In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report
import numpy as np

In [16]:
# Load the dataset and # Separate features and target
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
X_cleaned = df['clean_comment']
y_cleaned = df['category']

In [17]:
# Split the cleaned data into train and test sets (80-20 split)
X_train_cleaned, X_test_cleaned, y_train_cleaned, y_test_cleaned = train_test_split(X_cleaned, y_cleaned, test_size=0.2, random_state=42)

In [18]:
# Tokenize the sentences for Word2Vec training
X_train_tokenized = [sentence.split() for sentence in X_train_cleaned]
X_test_tokenized = [sentence.split() for sentence in X_test_cleaned]

In [19]:
# Train Word2Vec model
word2vec_model = Word2Vec(sentences=X_train_tokenized, vector_size=300, window=5, min_count=1, workers=4, sg=1)  # sg=1 for Skip-Gram model

In [20]:
# Generate the vector representation for each comment
def vectorize_comments(tokenized_comments, word2vec_model):
    vectorized_comments = []
    for tokens in tokenized_comments:
        vectors = [word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv]
        if len(vectors) > 0:
            vectorized_comments.append(np.mean(vectors, axis=0))  # Mean of all word vectors in the comment
        else:
            vectorized_comments.append(np.zeros(word2vec_model.vector_size))  # If no words match, use a zero vector
    return np.array(vectorized_comments)

In [21]:
# Vectorize train and test comments
X_train_word2vec = vectorize_comments(X_train_tokenized, word2vec_model)
X_test_word2vec = vectorize_comments(X_test_tokenized, word2vec_model)

In [22]:
best_model = LGBMClassifier(

    objective='multiclass',
    num_class=3,
    metric="multi_logloss",
    is_unbalance= True,
    class_weight= "balanced",
    reg_alpha= 0.1,  # L1 regularization
    reg_lambda= 0.003399270439473628,  # L2 regularization
    learning_rate= 0.08,
    max_depth= 10,
    n_estimators=327,
    min_child_samples=10
)

In [23]:
# Train the LightGBM model on Word2Vec embeddings
best_model.fit(X_train_word2vec, y_train_cleaned)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.034389 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 76500
[LightGBM] [Info] Number of data points in the train set: 29329, number of used features: 300
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


,boosting_type,'gbdt'
,num_leaves,31
,max_depth,10
,learning_rate,0.08
,n_estimators,327
,subsample_for_bin,200000
,objective,'multiclass'
,class_weight,'balanced'
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,10


In [24]:
# Make predictions on the test data
y_pred = best_model.predict(X_test_word2vec)

c:\Users\mukti\Desktop\python_DS\campus_x\reddit_sentiment_analysis\aws_env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [25]:
print(classification_report(y_test_cleaned, y_pred))

              precision    recall  f1-score   support

          -1       0.46      0.50      0.48      1647
           0       0.72      0.74      0.73      2510
           1       0.70      0.66      0.68      3176

    accuracy                           0.65      7333
   macro avg       0.63      0.63      0.63      7333
weighted avg       0.65      0.65      0.65      7333



In [26]:
###TEST AGAIN WITH HPT

import pandas as pd
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report
import numpy as np
import optuna
from sklearn.model_selection import cross_val_score

# Load the dataset
dataset = pd.read_csv('reddit_preprocessing.csv')

# Drop rows with NaN values in 'clean_comment'
cleaned_dataset = dataset.dropna()

# Separate features and target
X_cleaned = cleaned_dataset['clean_comment']
y_cleaned = cleaned_dataset['category']

# Split the cleaned data into train and test sets (80-20 split)
X_train_cleaned, X_test_cleaned, y_train_cleaned, y_test_cleaned = train_test_split(
    X_cleaned, y_cleaned, test_size=0.2, random_state=42)

# Tokenize the sentences for Word2Vec training
X_train_tokenized = [sentence.split() for sentence in X_train_cleaned]
X_test_tokenized = [sentence.split() for sentence in X_test_cleaned]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=X_train_tokenized, vector_size=100, window=5, min_count=1, workers=4, sg=1)  # sg=1 for Skip-Gram model

# Generate the vector representation for each comment
def vectorize_comments(tokenized_comments, word2vec_model):
    vectorized_comments = []
    for tokens in tokenized_comments:
        vectors = [word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv]
        if len(vectors) > 0:
            vectorized_comments.append(np.mean(vectors, axis=0))  # Mean of all word vectors in the comment
        else:
            vectorized_comments.append(np.zeros(word2vec_model.vector_size))  # If no words match, use a zero vector
    return np.array(vectorized_comments)

# Vectorize train and test comments
X_train_word2vec = vectorize_comments(X_train_tokenized, word2vec_model)
X_test_word2vec = vectorize_comments(X_test_tokenized, word2vec_model)

# Encode the target labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_cleaned)
y_test_encoded = label_encoder.transform(y_test_cleaned)

# Objective function for Optuna hyperparameter tuning
def objective(trial):
    # Suggest hyperparameters to be tuned
    params = {
        "objective": "multiclass",
        "num_class":3,
        "learning_rate": trial.suggest_float("learning_rate",1e-3, 1e-1),
        "n_estimators" : trial.suggest_int("n_estimators",50,500),
        "min_child_samples": trial.suggest_int('min_child_samples', 10, 200),
        "reg_lambda" : trial.suggest_float('reg_lambda', 1e-4, 50.0, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        "metric": "multi_logloss",
        "is_unbalance": True,
        "class_weight": "balanced",
    }

    # Initialize the LightGBM model with suggested parameters
    model = LGBMClassifier(**params)

    # Perform cross-validation to evaluate the model performance
    scores = cross_val_score(model, X_train_word2vec, y_train_encoded, cv=3, scoring='accuracy')

    # Return the mean accuracy score across folds
    return scores.mean()

# Create an Optuna study to optimize the objective function
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)  # Run 30 trials

# Print the best trial
print("Best trial:")
best_params = study.best_trial.params
print(best_params)

# Train the LightGBM model with the best parameters
best_model = LGBMClassifier(
    objective='multiclass',
    num_class=3,
    metric="multi_logloss",
    is_unbalance= True,
    class_weight= "balanced",    
    **best_params
)

best_model.fit(X_train_word2vec, y_train_encoded)

# Make predictions on the test data
y_pred = best_model.predict(X_test_word2vec)



[I 2025-11-19 08:28:02,445] A new study created in memory with name: no-name-0eb7a164-50d3-4ac3-84ad-49dafe174d06


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005724 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 19552, number of used features: 100
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


c:\Users\mukti\Desktop\python_DS\campus_x\reddit_sentiment_analysis\aws_env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006745 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 19553, number of used features: 100
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


c:\Users\mukti\Desktop\python_DS\campus_x\reddit_sentiment_analysis\aws_env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


...

c:\Users\mukti\Desktop\python_DS\campus_x\reddit_sentiment_analysis\aws_env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007757 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 19553, number of used features: 100
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


c:\Users\mukti\Desktop\python_DS\campus_x\reddit_sentiment_analysis\aws_env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-11-19 08:35:12,291] Trial 29 finished with value: 0.6518122719570593 and parameters: {'learning_rate': 0.09159982227616144, 'n_estimators': 391, 'min_child_samples': 128, 'reg_lambda': 0.0003497023643178789, 'max_depth': 16}. Best is trial 24 with value: 0.6527669491471042.


Best trial:
{'learning_rate': 0.08830686896476032, 'n_estimators': 441, 'min_child_samples': 161, 'reg_lambda': 0.0007619309880420349, 'max_depth': 11}
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010058 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 29329, number of used features: 100
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning]

c:\Users\mukti\Desktop\python_DS\campus_x\reddit_sentiment_analysis\aws_env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [27]:
# Print classification report
print(classification_report(y_test_encoded, y_pred))

              precision    recall  f1-score   support

           0       0.47      0.49      0.48      1647
           1       0.73      0.72      0.73      2510
           2       0.68      0.68      0.68      3176

    accuracy                           0.65      7333
   macro avg       0.63      0.63      0.63      7333
weighted avg       0.65      0.65      0.65      7333

